<a href="https://colab.research.google.com/github/kurtlee1984/Modification_code/blob/main/Scaled_YOLOv4_Pytorch_modificaton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 將 Colab 與 google_drive 綁定
# 連接到指定帳號,並複製授權碼貼上

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 當前設備顯卡參數列表明細:

!nvidia-smi

In [ ]:
# 從 Github 下載 Scaled_YOLOv4 資料庫
!git clone https://github.com/roboflow-ai/ScaledYOLOv4.git  # clone repo
%cd /content/ScaledYOLOv4/

# 可選擇版本 yolov4-large 或 yolov4-CSP 架構, 可擇一使用

# 確認 yolov4-large branch
!git checkout yolov4-large

# 確認 yolov4-CSP branch
# !git checkout yolov4-CSP

In [ ]:
# 導入需求套件和設備配置確認
import torch
print('Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
# 安裝並啟動 cuda 顯卡優化套件
%cd /content/
!git clone https://github.com/JunnYu/mish-cuda
%cd mish-cuda
!python setup.py build install

In [ ]:
# 下載安裝 PyYAM (為 python 的 yaml 檔案的解析套件)
!pip install -U PyYAML

In [ ]:
# 轉換當前路徑目錄
# 下載資料集,並於 google_drive 取得壓縮檔的 id
# id 取得方法,到 google_drive 指定壓縮檔,滑鼠右鍵選擇取得連結,更改權限由限制->知道連結的使用者,複製連結在筆記本上
# https://drive.google.com/file/d/這裡就是壓縮檔的id/view?usp=sharing

%cd /content
!gdown --id '159gaoK_65Qfbez5llMrkvHqUR3KYOFl7' -O dataset.zip

In [ ]:
# 解壓縮,注意壓縮檔名稱需與代碼一致
# 解壓縮完,於右邊選擇檔案,看有無已解壓縮的資料夾

!unzip -q dataset.zip; rm dataset.zip

In [ ]:
# 確定檔案位置和類別數

%cd /content
%cat data.yaml

/content
train: ../train/images
val: ../valid/images

nc: 33
names: ['bacon', 'bagel', 'baguette', 'baozi', 'burger', 'cereal', 'chips', 'cong_you_bing', 'corn_soup', 'croissant', 'danbing', 'drinks', 'fantuan', 'fried_baozi', 'fried_egg', 'guotie', 'ham', 'hash_brown', 'hotdog', 'pancake', 'radish_cake', 'salad', 'sandwich', 'shaobing', 'sliced_bread', 'steamed_bread', 'tea_egg', 'teppanyaki_noodle', 'triangle_rice_ball', 'waffle', 'water_rice', 'xiaolongbao', 'you_tiao']

In [ ]:
# 根據 data.yaml 來定義訓練的類別數

import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
# 讀取ScaledYOLOv4資料夾內yolov4的模型框架

%cat /content/ScaledYOLOv4/models/yolov4-csp.yaml

In [ ]:
# 以自己所需求的 epoch 數量去訓練S caledYOLOv4, 可自行修改所需要的 epochs 數量,這裡是1000epochs
# 參數說明: img(輸入的圖片尺寸), batch(batch 大小), epochs(epochs數量), data(.yaml的路徑),cfg(模型的架構),
#            weights(權重,可修改成自己已訓練的權重路徑),name(結果的名稱), nosave(只存最後的checkpoint), cache(內存與圖像高速緩存)

%%time
%cd /content/ScaledYOLOv4/
!python train.py --img 416 --batch 16 --epochs 1000 --data '../data.yaml' --cfg ./models/yolov4-csp.yaml --weights '' --name yolov4-csp-results  --cache

In [ ]:
# 使用 tensorboard 可視化套件
# 讀取"runs"資料夾下的結果

%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# 對於某些較老舊的圖表,如果 tensorboard 無法顯示,請使用以下套件繪製

from utils.general import plot_results  
plot results.txt as results.png

In [ ]:
# 導入繪圖與顯示套件
from IPython.display import Image, display

# 圖片路徑(如果不對記得修改), 繪圖參數
display(Image('/content/ScaledYOLOv4/runs/exp0_yolov4-csp-results/results.png'))  # view results.png

In [ ]:
# 顯示正確 Test 的照片和其標註的 label
# 圖片路徑(如果不對記得修改), 繪圖參數

print("GROUND TRUTH TRAINING DATA:")
Image(filename='/content/ScaledYOLOv4/runs/exp0_yolov4-csp-results/test_batch0_gt.jpg', width=900)

In [ ]:
# 顯示augment後test的照片和其標註的label
# 圖片路徑(如果不對記得修改), 繪圖參數

print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/content/ScaledYOLOv4/runs/exp0_yolov4-csp-results/train_batch0.jpg', width=900)

In [ ]:
# 已訓練完的權重會存在 runs 資料夾, 並檢視檔案與目錄的狀態

%ls runs/

In [ ]:
# 檢視檔案與目錄的狀態

%ls ./runs/exp0_yolov4-csp-results/weights

In [ ]:
# inference 存放 test 資料夾下的照片
# 使用訓練時最好的權重, best.pt (需要注意路徑是否正確)

%cd /content/ScaledYOLOv4/
!python detect.py --weights ./runs/exp0_yolov4-csp-results/weights/best_yolov4-csp-results.pt --img 416 --conf 0.4 --source ../test/images

In [ ]:
# 顯示出 inference 後的圖片(需要注意路徑是否正確)

import glob
from IPython.display import Image, display

for imageName in glob.glob('./inference/output/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

In [ ]:
#下載權重檔案到google_drive

from google.colab import files
files.download('./runs/exp0_yolov4-csp-results/weights/best_yolov4-csp-results.pt')